In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

base_model = keras.applications.Xception(
    weights='imagenet',
    include_top=True,
    input_shape=(299, 299, 3)
)

raw_dataset = tf.data.TFRecordDataset(['birds-vs-squirrels-train.tfrecords'])
feature_description = {
    'image': tf.io.FixedLenFeature([], tf.string),
    'label': tf.io.FixedLenFeature([], tf.int64)
}

def parse_examples(serialized_examples):
    examples = tf.io.parse_example(serialized_examples, feature_description)
    targets = examples.pop('label')
    images = tf.image.resize_with_pad(
        tf.cast(
            tf.io.decode_jpeg(examples['image'], channels=3),
            tf.float32
        ),
        299,
        299
    )
    return images, targets



91884032/91884032 [==============================] - 2s 0us/step


In [7]:
dataset = raw_dataset.map(parse_examples, num_parallel_calls=tf.data.experimental.AUTOTUNE)
dataset = dataset.batch(16)

# Calculate the dataset size manually
dataset_size = tf.data.experimental.cardinality(dataset).numpy()

# Split the dataset into train and test sets
train_size = int(0.6 * dataset_size)
valid_size = int(0.2 * dataset_size)
test_size = dataset_size - train_size - valid_size

trainset = dataset.take(train_size)
validset = dataset.skip(train_size).take(valid_size)
testset = dataset.skip(train_size + valid_size)

# Print the number of samples in each set
print("Train set size:", train_size)
print("Validation set size:", valid_size)
print("Test set size:", test_size)

def preprocessWithAspectRatio(image, label):
    resized_image = tf.image.resize_with_pad(image, 299, 299)
    reshaped_image = tf.reshape(resized_image, (-1, 299, 299, 3))
    final_image = keras.applications.xception.preprocess_input(reshaped_image)
    return final_image, label


trainPipe = trainset.map(preprocessWithAspectRatio, num_parallel_calls=tf.data.experimental.AUTOTUNE).cache() 
validPipe = validset.map(preprocessWithAspectRatio, num_parallel_calls=tf.data.experimental.AUTOTUNE).cache() 
testPipe = testset.map(preprocessWithAspectRatio, num_parallel_calls=tf.data.experimental.AUTOTUNE).cache()

flatten = keras.layers.Flatten()(base_model.output)
dense = keras.layers.Dense(128, activation='relu')(flatten)
output = keras.layers.Dense(3, activation='softmax')(dense)
model = keras.models.Model(inputs=base_model.input, outputs=output)

for layer in base_model.layers: 
    layer.trainable = False 

checkpoint_cb = keras.callbacks.ModelCheckpoint('preprocessing.h5', save_best_only=True) 
earlyStop_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True) 
learning_rate = 0.5
optimizer = keras.optimizers.SGD(learning_rate=learning_rate) 

model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
model.fit(trainPipe, validation_data=validPipe, epochs=5, callbacks=[checkpoint_cb, earlyStop_cb])





Train set size: -1
Validation set size: 0
Test set size: -1
Epoch 1/5


NotFoundError: ignored